In [3]:
import warnings
warnings.filterwarnings(action='ignore')

import os
import pandas as pd
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
import IPython.display as ipd
import time

In [4]:
import torch
import torchvision
from torch import Tensor, nn, optim
from torch.utils.data import Dataset, DataLoader

import torch.nn.functional as F
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from adamp import AdamP
from torchsummary import summary

import albumentations as A
from albumentations.pytorch import ToTensor
from efficientnet_pytorch import EfficientNet

## 시드 고정

In [5]:
import random
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = True  
seed_everything()

## 데이터 불러오기

In [6]:
train = pd.read_csv('./train/train_data.csv')
test = pd.read_csv('./test/test/test_data.csv')

In [7]:
train.columns = ['file_name','label']

In [8]:
train.head()

,file_name,label
0,train0001.png,8
1,train0002.png,8
2,train0003.png,8
3,train0004.png,8
4,train0005.png,8


In [9]:
test.head()

,file_name
0,idx0001.png
1,idx0002.png
2,idx0003.png
3,idx0004.png
4,idx0005.png


In [10]:
def get_transform(mode='train'):
    # Data Augmentation
    # 마가리따님의 '파이토치 데이터 증강법 (점수:0.835)' 참고 : https://dacon.io/competitions/official/235838/codeshare/3734?page=1&dtype=recent
    
    t = list()
    t.append(transforms.ToTensor())
    
    if mode == 'train':
        t.append(transforms.RandomAffine(degrees=10, translate=(0.1,0.1)))

    t.append(transforms.Normalize(mean=(.5),std=(.5)))
    
    return transforms.Compose(t)

In [27]:
class CustomDataset(torch.utils.data.Dataset): 
    def __init__(self,df,path,option,augmentation=None):
        self.df = df
        self.option = option
        self.augmentation = augmentation
        
        self.path = path

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx): 
        file_path  = self.df.iloc[idx,0]
        image=  Image.open(os.path.join(self.path,file_path)).convert('RGB')
        #image = image.resize((100,100))
        image = np.array(image)
        
        #image = transforms.ToTensor()(image=image)['image']
        
        
        if self.augmentation is not None:
            image = self.augmentation(image=image)['image']
        
        
        if self.option =='train':
            label = self.df.iloc[idx,1]
            label = torch.tensor(label, dtype=torch.int64)
            return image, label
        
        return image
    

In [28]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(batch_size, 28, 28, 2)
        #    Conv     -> (batch_size, 28, 28, 16)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 16, kernel_size=3, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(16),
            torch.nn.Dropout(p=0.3))
        
        # 두번째층
        # ImgIn shape=(batch_size, 28, 28, 16)
        #    Conv      ->(batch_size, 28, 28, 32)
        #    Pool      ->(batch_size, 9, 9, 32)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(32),
            torch.nn.Conv2d(32, 32, kernel_size=5, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(32),
            torch.nn.Conv2d(32, 32, kernel_size=5, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(32),
            torch.nn.Conv2d(32, 32, kernel_size=5, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(32),
            torch.nn.MaxPool2d(kernel_size=3, stride=3),
            torch.nn.Dropout(p=0.3))
        
        # 세번째층
        # ImgIn shape=(batch_size, 9, 9, 32)
        #    Conv      ->(batch_size, 9, 9, 64)
        #    Pool      ->(batch_size, 3, 3, 64)        
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(64),
            torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(64),
            torch.nn.MaxPool2d(kernel_size=3, stride=3),
            torch.nn.Dropout(p=0.3))
        
        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(3*3*64, 10, bias=True)

        torch.nn.init.xavier_uniform_(self.fc.weight) # fc 가중치 초기화

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten
        out = self.fc(out)
        return out

In [29]:
class MnistModel(nn.Module):
    def __init__(self,):
        super().__init__()
        #self.conv2d = nn.Conv2d(3, 3, 3, stride=1)
        self.model = torchvision.models.resnet18(pretrained = False)
        #self.model = EfficientNet.from_name('efficientnet-b1')
        self.dropout = nn.Dropout(0.5)
        
        self.classifier = nn.Linear(1000, 10)

    def forward(self, images):
        #outputs = self.conv2d(images)
        outputs = self.model(images)
        outputs = self.dropout(outputs)
        outputs = self.classifier(outputs)
        return outputs

In [32]:
class EarlyStopping:
    # 참고: https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py
    
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, k_num=0):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        
        self.k_num = k_num

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            #self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
#             print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            #self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path + f'/best_{k_num}.pt')
        self.val_loss_min = val_loss

In [83]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 5)
folds=[]
for trn_idx,val_idx in skf.split(train['file_name'],train['label']):
    folds.append((trn_idx,val_idx))

In [110]:
num_epochs = 50
best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장
k_num = 0

for i,fold in enumerate(range(5)):
    print('===============',i+1,'fold start===============')
    
    model = CNN().to('cuda')
    k_num += 1
    early_stopping = EarlyStopping(patience = 300, verbose = True, k_num = k_num)
    
    #optimizer = AdamP(model.parameters(), lr=1e-3, )
    optimizer = optim.SGD(model.parameters(), lr=1e-3, )
    criterion = nn.CrossEntropyLoss()
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size = 5,
                                                gamma = 0.9)
        
    
    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]
    
    train_data = train.loc[trn_idx]
    val_data = train.loc[valid_idx]
    
    train_transforms = A.Compose([
    A.Normalize(),
    ToTensor()
    ])

    test_transforms = A.Compose([
        A.Normalize(),
        ToTensor()
        ])

    train_dataset = CustomDataset(train_data,'train/','train',train_transforms)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)

    valid_dataset = CustomDataset(val_data,'train/','train',test_transforms)
    valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=0)
    
    
    valid_acc_max = 0.85
    valid_loss_min = 0.4
    for epoch in range(num_epochs):

        for i, (images, targets) in enumerate(train_loader):
            model.train()
            images, targets = images.to('cuda'), targets.to('cuda')

            optimizer.zero_grad()
            outputs = model(images)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()  

            if (i+1) % 20 == 0:
                print(f'Epoch: {epoch} - Loss: {loss:.6f}')


        val_loss = []
        val_acc = []
        for i, (images, targets) in enumerate(valid_loader):
            model.eval()
            images, targets = images.to('cuda'), targets.to('cuda')

            with torch.no_grad():
                outputs = model(images)
                valid_loss = criterion(outputs,targets).cpu().detach().numpy()

                preds = torch.argmax(outputs,axis = 1)
                preds = preds.cpu().detach().numpy()

                targets = targets.cpu().detach().numpy()
                batch_acc = (preds==targets).mean()



                val_loss.append(valid_loss)
                val_acc.append(batch_acc)

        val_loss = np.mean(val_loss)
        val_acc = np.mean(val_acc)

        print(f'Epoch: {epoch} - valid Loss: {val_loss:.6f} - valid_acc : {val_acc:.6f}')
        early_stopping(float(val_loss), model)
        if early_stopping.early_stop:
            print("Early stopping!!")
            break       
        
        '''
         if valid_acc_max < val_acc:
            valid_acc_max = val_acc
            best_models.append(model)
            print('model save, model val acc : ',val_acc)
            print('best_models size : ',len(best_models))
            '''


        if valid_loss_min > val_loss:
            valid_loss_min = val_loss
            best_models.append(model)
            
    # Learning rate 조절
    lr_scheduler.step()
    
  
    
        
        

=============== 1 fold start===============
Epoch: 0 - Loss: 3.242912
Epoch: 0 - Loss: 2.826890
Epoch: 0 - Loss: 1.860245
Epoch: 0 - valid Loss: 0.905390 - valid_acc : 0.681641
Epoch: 1 - Loss: 1.255856
Epoch: 1 - Loss: 1.245807
Epoch: 1 - Loss: 1.273116
Epoch: 1 - valid Loss: 0.369037 - valid_acc : 0.884766
Epoch: 2 - Loss: 0.787956
Epoch: 2 - Loss: 0.715223
Epoch: 2 - Loss: 0.538051
Epoch: 2 - valid Loss: 0.250748 - valid_acc : 0.925781
Epoch: 3 - Loss: 0.956386
Epoch: 3 - Loss: 0.580319
Epoch: 3 - Loss: 0.521698
Epoch: 3 - valid Loss: 0.207277 - valid_acc : 0.942383
Epoch: 4 - Loss: 0.511788
Epoch: 4 - Loss: 0.581196
Epoch: 4 - Loss: 0.409695
Epoch: 4 - valid Loss: 0.161027 - valid_acc : 0.952539
Epoch: 5 - Loss: 0.356975
Epoch: 5 - Loss: 0.411231
Epoch: 5 - Loss: 0.369376
Epoch: 5 - valid Loss: 0.134437 - valid_acc : 0.961914
Epoch: 6 - Loss: 0.398980
Epoch: 6 - Loss: 0.382625
Epoch: 6 - Loss: 0.302937
Epoch: 6 - valid Loss: 0.120492 - valid_acc : 0.961914
Epoch: 7 - Loss: 0.478519

In [111]:
test_transforms = A.Compose([
    A.Normalize(),
    ToTensor()
])

test_dataset = CustomDataset(test,'test/test','test',test_transforms)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

In [112]:
len(best_models)

178

In [113]:
preds = []
for idx,model in enumerate(best_models): 
    print(idx+1, '번째 모델 예측 진행중')
    model = model
    model.eval()
    y_pred = []
    with torch.no_grad():
        for i, d in enumerate(test_loader):
            inputs = d.to('cuda')
            outputs = model(inputs).detach().cpu().numpy()
            y_pred.extend(outputs.argmax(axis=1).astype(int))
            
    preds.append(y_pred)




1 번째 모델 예측 진행중
2 번째 모델 예측 진행중
3 번째 모델 예측 진행중
4 번째 모델 예측 진행중
5 번째 모델 예측 진행중
6 번째 모델 예측 진행중
7 번째 모델 예측 진행중
8 번째 모델 예측 진행중
9 번째 모델 예측 진행중
10 번째 모델 예측 진행중
11 번째 모델 예측 진행중
12 번째 모델 예측 진행중
13 번째 모델 예측 진행중
14 번째 모델 예측 진행중
15 번째 모델 예측 진행중
16 번째 모델 예측 진행중
17 번째 모델 예측 진행중
18 번째 모델 예측 진행중
19 번째 모델 예측 진행중
20 번째 모델 예측 진행중
21 번째 모델 예측 진행중
22 번째 모델 예측 진행중
23 번째 모델 예측 진행중
24 번째 모델 예측 진행중
25 번째 모델 예측 진행중
26 번째 모델 예측 진행중
27 번째 모델 예측 진행중
28 번째 모델 예측 진행중
29 번째 모델 예측 진행중
30 번째 모델 예측 진행중
31 번째 모델 예측 진행중
32 번째 모델 예측 진행중
33 번째 모델 예측 진행중
34 번째 모델 예측 진행중
35 번째 모델 예측 진행중
36 번째 모델 예측 진행중
37 번째 모델 예측 진행중
38 번째 모델 예측 진행중
39 번째 모델 예측 진행중
40 번째 모델 예측 진행중
41 번째 모델 예측 진행중
42 번째 모델 예측 진행중
43 번째 모델 예측 진행중
44 번째 모델 예측 진행중
45 번째 모델 예측 진행중
46 번째 모델 예측 진행중
47 번째 모델 예측 진행중
48 번째 모델 예측 진행중
49 번째 모델 예측 진행중
50 번째 모델 예측 진행중
51 번째 모델 예측 진행중
52 번째 모델 예측 진행중
53 번째 모델 예측 진행중
54 번째 모델 예측 진행중
55 번째 모델 예측 진행중
56 번째 모델 예측 진행중
57 번째 모델 예측 진행중
58 번째 모델 예측 진행중
59 번째 모델 예측 진행중
60 번째 모델 예측 진행중
61 번째 모델 예측 진행중
62 번째 모델 예측 진행중
63 번째 모델 예측 진행중
6

In [106]:
from collections import Counter
np_pred = np.array(preds).T

pred = []
for i in range(5000):
    cnt = Counter(np_pred[i])
    pred.append(cnt.most_common()[0][0])
#pred

In [107]:
submission = pd.read_csv('sample_submission.csv')
submission['label'] = pred

In [108]:
submission.to_csv('sub_211125.csv',index = False)

In [109]:
submission['label'].value_counts()

1    663
6    587
0    555
9    531
7    526
2    467
5    467
3    461
4    449
8    294
Name: label, dtype: int64